# Bài 8: Keras, Regression

Trong bài thực hành này, chúng ta sẽ 
- Làm quen với Tensorflow thông qua các bài toán Regression


## 1. Dự đoán giá nhà (Lab 01)

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("House_Price.csv")   
df.head()

,width,length,price
0,28,97,3.350184e+08
1,21,62,2.298295e+08
2,27,64,2.603552e+08
3,5,92,2.064708e+08
4,13,84,2.339808e+08


In [2]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
width,100.0,1.691000e+01,7.254528e+00,5.000000e+00,1.100000e+01,1.600000e+01,2.300000e+01,2.900000e+01
length,100.0,5.977000e+01,2.278483e+01,2.000000e+01,3.900000e+01,6.050000e+01,7.900000e+01,9.700000e+01
price,100.0,2.046623e+08,6.439491e+07,7.257961e+07,1.583727e+08,1.951909e+08,2.513378e+08,3.377315e+08


In [3]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.1)

## dùng width và length để đoán price
X_train = df_train[['width', 'length']].values    
X_test = df_test[['width', 'length']].values

X_means = np.mean(X_train, axis=0)
X_stds = np.std(X_train, axis=0)
X_train = (X_train - X_means)/X_stds
X_test = (X_test - X_means)/X_stds

y_train = df_train['price'].values
y_test = df_test['price'].values

y_means = np.mean(y_train, axis=0)
y_stds = np.std(y_train, axis=0)


In [4]:
X_train.shape, y_train.shape

((90, 2), (90,))

In [5]:
X_test.shape, y_test.shape

((10, 2), (10,))

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(2,))    ## tạo một layer Input, mỗi sample có input là 1 vector độ dài 2
dense_1 = Dense(units=10, activation='relu')(inputs)   ## tạo một layer gồm 10 units, hàm kích hoạt relu
outputs = Dense(units=1, activation=None)(dense_1)   ## tạo layer output gồm 1 units, tượng trưng cho giá nhà, vì giá nhà thực nên không để hàm kích hoạt
model = Model(inputs=inputs, outputs=outputs)    ## tạo model với input và output layer đã xây dựng

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1000.0),
             loss='mean_squared_error')      ## khai báo optimizer và loss cho model
model.summary()           

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 10)                30        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=80)

Epoch 1/10
2/2 [==============================] - 1s 318ms/step - loss: 46466435131637760.0000 - val_loss: 34626885333811200.0000
Epoch 2/10
2/2 [==============================] - 0s 34ms/step - loss: 36216554377620136.0000 - val_loss: 14734569282469888.0000
Epoch 3/10
2/2 [==============================] - 0s 29ms/step - loss: 13516507967389696.0000 - val_loss: 4049552059400192.0000
Epoch 4/10
2/2 [==============================] - 0s 33ms/step - loss: 8124140020039680.0000 - val_loss: 4166196391837696.0000
Epoch 5/10
2/2 [==============================] - 0s 33ms/step - loss: 5584187271326379.0000 - val_loss: 921753870139392.0000
Epoch 6/10
2/2 [==============================] - 0s 35ms/step - loss: 1545845926614357.2500 - val_loss: 2202765514244096.0000
Epoch 7/10
2/2 [==============================] - 0s 33ms/step - loss: 3542185156804608.0000 - val_loss: 2751286256599040.0000
Epoch 8/10
2/2 [==============================] - 0s 34ms/step - loss: 3897012269350912.0000 - val_loss: 1

In [8]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, r2_score, accuracy_score

y_train_pred = model.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train,y_train_pred))

r2 = r2_score(y_train,y_train_pred)

print('RMSE: ', rmse)
print('R2: ', r2)

RMSE:  39689517.81680585
R2:  0.6151668627201496


In [9]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = (np.sqrt(mse))
mv = explained_variance_score(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model testing performance:")
print("--------------------------")
print("Mean Absolute Error is", round(mae,1))
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("Variance explained by model", round(mv*100,5), "%")

Model testing performance:
--------------------------
Mean Absolute Error is 23595363.5
RMSE is 30947556.186380286
R2 score is 0.7677577731224416
Variance explained by model 87.89794 %


## 2. Bài tập: COVID-19 Retweet Prediction

Link: https://www.cikm2020.org/covid-19-retweet-prediction-challenge/

Dự đoán số lượng retweet của một tweet dựa vào các metadata như số #friends, #followers, ... của người đăng, thời gian, hashtags của tweet.

Download: https://drive.google.com/drive/folders/1eZwesVW9RcW_0ZJt0ztGyhTlISx3USn5?usp=sharing

In [10]:
df_train = pd.read_csv("df_train.csv")
df_train.head()

,timestamp,#followers,#friends,#favorites,hashtags,sentiment_1,sentiment_2,#retweet
0,Sun Mar 29 07:40:34 +0000 2020,6,259,0,null;,3,-1,0
1,Sun Mar 22 12:06:15 +0000 2020,5133,5255,0,null;,3,-1,0
2,Sun Mar 15 20:01:23 +0000 2020,120323,4912,6,null;,3,-2,5
3,Wed Nov 13 19:31:39 +0000 2019,9,140,0,null;,1,-1,0
4,Mon Apr 27 14:31:02 +0000 2020,12121,568,28,OpenMISafely: TrustMI,2,-1,10


In [11]:
df_train.isnull().sum()

timestamp         0
#followers        0
#friends          0
#favorites        0
hashtags       3166
sentiment_1       0
sentiment_2       0
#retweet          0
dtype: int64

In [12]:
df_test = pd.read_csv("df_test.csv")
df_test.head()

,timestamp,#followers,#friends,#favorites,hashtags,sentiment_1,sentiment_2,#retweet
0,Sat Apr 18 16:48:10 +0000 2020,4922,2945,0,null;,1,-1,0
1,Thu Mar 19 17:04:47 +0000 2020,749,262,0,null;,1,-1,0
2,Fri Mar 13 19:22:16 +0000 2020,592046,15755,215,COVIDー19,1,-3,59
3,Tue Apr 07 17:23:43 +0000 2020,130,116,0,null;,1,-3,0
4,Sun Jan 19 11:41:23 +0000 2020,360,70,0,null;,2,-1,0


In [13]:
df_test.isnull().sum()

timestamp        0
#followers       0
#friends         0
#favorites       0
hashtags       324
sentiment_1      0
sentiment_2      0
#retweet         0
dtype: int64

In [14]:
df_train = df_train.dropna()

In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7333205 entries, 0 to 7336370
Data columns (total 8 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   timestamp    object
 1   #followers   int64 
 2   #friends     int64 
 3   #favorites   int64 
 4   hashtags     object
 5   sentiment_1  int64 
 6   sentiment_2  int64 
 7   #retweet     int64 
dtypes: int64(6), object(2)
memory usage: 503.5+ MB


In [16]:
df_test = df_test.dropna()

In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 814829 entries, 0 to 815152
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   timestamp    814829 non-null  object
 1   #followers   814829 non-null  int64 
 2   #friends     814829 non-null  int64 
 3   #favorites   814829 non-null  int64 
 4   hashtags     814829 non-null  object
 5   sentiment_1  814829 non-null  int64 
 6   sentiment_2  814829 non-null  int64 
 7   #retweet     814829 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 55.9+ MB


In [18]:
df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
df_train.head()

KeyboardInterrupt: 

## Bài tập
Sử dụng một vài hoặt tất cả các cột trong số các cột timestamp, #followers, #friends, #favorites, hashtags, sentiment_1, sentiment_2 để dự đoán cột #retweet:
- Dùng model machine learning bất kì (nên dùng neural network)
- Đạt Mean Squared Logarithmic Error (https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredLogarithmicError) trên tập test ít nhất là 0.25 mới được đầy đủ điểm

## Nộp bài
- Code và chạy kết quả lưu vào file notebook NMMH_TH8_MSSV.ipynb (notebook phải có kết quả chạy nếu ko xem như chưa làm)
- Nén thành file NMMH_TH8_MSSV.rar (.zip) và nộp về: dinhvietcuong1996@gmail.com
- Deadline: 23g59 thứ 3 ngày 28/07/2020. Nộp trễ bị chia đôi số điểm.